In [95]:
import pyterrier as pt
import pandas as pd
import pickle
import nltk
import numpy as np

In [96]:
pd.options.mode.copy_on_write = True

In [97]:
df_jahreswechsel = pd.read_json('jahreswechsel.json')

In [98]:
df_jahreswechsel

,id,tags,title,user,description,date,changeDate,url,pub-type,year,...,volume,pages,bibtexKey,pid,booktitle,series,publisher,editors,isbn,search_term
0,https://www.bibsonomy.org/bibtex/2a9f754572c68...,"[Rom, Silvester]","Silvester I., römischer Bischof",avs,,2009-01-20 09:50:15,2009-01-20 09:50:16,,article,2004,...,VII,1321--1322,2004b,None,None,None,None,None,None,Silvester
1,https://www.bibsonomy.org/bibtex/241dfc656397b...,"[Purchasing, the, Price, and, Decision, Promot...",The Effect of Price and Promotion on the Purch...,ijtsrd,,2024-03-08 08:00:25,2024-03-08 08:00:25,https://www.ijtsrd.com/economics/market-econom...,article,2024,...,8,636-657,HastieEtAl2008,None,None,None,None,None,None,Silvester
2,https://www.bibsonomy.org/bibtex/235bf77da240d...,"[Location,, Rates,, Cooperative, Service, the,...","The Influence of Location, Credit Interest Rat...",ijtsrd,,2024-03-08 08:02:00,2024-03-08 08:02:00,https://www.ijtsrd.com/economics/development-e...,article,2024,...,8,658-675,journals/is/Formica03,http://dx.doi.org/10.1016/S0306-4379(02)00010-8,None,None,None,None,None,Silvester
3,https://www.bibsonomy.org/bibtex/23674c2f2342a...,[imported],"Silvester I., r�mischer Bischof",avs,,2009-01-20 09:49:38,2009-01-20 09:49:40,,article,2004,...,VII,1321--1322,2004b,None,None,None,None,None,None,Silvester
4,https://www.bibsonomy.org/bibtex/289e0b4f233d1...,[dnb],Gerbert (Silvester II.) als Persönlichkeit,genealogie,,2016-05-31 03:35:04,2016-05-31 03:35:04,,phdthesis,1928,...,None,None,eichengrun1928gerbert,None,None,None,None,None,None,Silvester
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,https://www.bibsonomy.org/bibtex/209424eb679de...,[dblp],Solving Multidimensional 0-1 Knapsack Problem ...,dblp,,2018-11-14 00:00:00,2019-05-15 17:34:36,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7335,72-86,conf/iccsa/AzadRF12,https://www.wikidata.org/entity/Q57573541,ICCSA (3),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31136-9,Neujahr
1953,https://www.bibsonomy.org/bibtex/210805c159439...,[dblp],Axis-Parallel Dimension Reduction for Biometri...,dblp,,2017-05-24 00:00:00,2019-05-15 17:32:59,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7333,188-197,conf/iccsa/AhmadianG12,https://doi.org/10.1007/978-3-642-31125-3_15,ICCSA (1),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31124-6,Neujahr
1954,https://www.bibsonomy.org/bibtex/2935b7a107d2a...,[dblp],A Review on Delaunay Refinement Techniques.,dblp,,2017-05-24 00:00:00,2019-05-15 17:32:59,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7333,172-187,conf/iccsa/Oliveira12,https://doi.org/10.1007/978-3-642-31125-3_14,ICCSA (1),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31124-6,Neujahr
1955,https://www.bibsonomy.org/bibtex/29432427bc781...,[dblp],Theoretical and Experimental Study of the Ener...,dblp,,2018-11-02 00:00:00,2019-05-15 17:32:59,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7333,316-330,conf/iccsa/RosiCFMPV12,https://doi.org/10.1007/978-3-642-31125-3_25,ICCSA (1),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31124-6,Neujahr


In [99]:
df_jahreswechsel.shape

(1957, 21)

In [100]:
df_jahreswechsel_filtered = df_jahreswechsel.dropna(axis=1, thresh=len(df_jahreswechsel)*0.5)

In [101]:
df_jahreswechsel_filtered.shape

(1957, 19)

In [102]:
df_jahreswechsel_filtered.columns

Index(['id', 'tags', 'title', 'user', 'description', 'date', 'changeDate',
       'url', 'pub-type', 'year', 'authors', 'pages', 'bibtexKey', 'pid',
       'booktitle', 'publisher', 'editors', 'isbn', 'search_term'],
      dtype='object')

In [103]:
df_jahreswechsel_filtered = df_jahreswechsel_filtered.drop(columns=['isbn'])

In [104]:
df_jahreswechsel_filtered.columns

Index(['id', 'tags', 'title', 'user', 'description', 'date', 'changeDate',
       'url', 'pub-type', 'year', 'authors', 'pages', 'bibtexKey', 'pid',
       'booktitle', 'publisher', 'editors', 'search_term'],
      dtype='object')

In [105]:
if not pt.java.started():
    pt.init()

In [106]:
df_jahreswechsel_filtered['docno'] = df_jahreswechsel_filtered['id']
df_jahreswechsel_filtered['text'] = df_jahreswechsel_filtered['title']

In [107]:
df_jahreswechsel_filtered.columns

Index(['id', 'tags', 'title', 'user', 'description', 'date', 'changeDate',
       'url', 'pub-type', 'year', 'authors', 'pages', 'bibtexKey', 'pid',
       'booktitle', 'publisher', 'editors', 'search_term', 'docno', 'text'],
      dtype='object')

In [108]:
jw_dict = df_jahreswechsel_filtered.to_dict(orient='records')

In [109]:
index_folder = './jw_index_mult'

index_fields = ['docno', 'text', 'tags', 'user', 'description', 'bibtexKey', 'booktitle']

indexer = pt.IterDictIndexer(index_folder, meta={'docno': 200,'text': 4096}, overwrite=True)
index_ref = indexer.index(jw_dict, fields=index_fields)

print(f'Indexing-Folder ist {index_folder}.')

Indexing-Folder ist ./jw_index_mult.


In [110]:
index = pt.IndexFactory.of(index_ref)

In [111]:
print(index.getCollectionStatistics().toString())

Number of documents: 1957
Number of terms: 5282
Number of postings: 31561
Number of fields: 7
Number of tokens: 36160
Field names: [docno, text, tags, user, description, bibtexKey, booktitle]
Positions:   false



In [112]:
for kv in index.getLexicon():
    print(f"{kv.getKey()} --> {kv.getValue().toString()}")

0 --> term440 Nt=6 TF=9 maxTF=4 @{0 0 0} TFf=0,9,0,0,0,0,0
0018 --> term5252 Nt=1 TF=1 maxTF=1 @{0 18 2} TFf=1,0,0,0,0,0,0
01 --> term1020 Nt=1 TF=1 maxTF=1 @{0 22 0} TFf=0,0,1,0,0,0,0
01a55 --> term1368 Nt=1 TF=1 maxTF=1 @{0 25 0} TFf=0,0,1,0,0,0,0
01a60 --> term1366 Nt=1 TF=1 maxTF=1 @{0 28 0} TFf=0,0,1,0,0,0,0
03 --> term1360 Nt=1 TF=1 maxTF=1 @{0 31 0} TFf=0,0,1,0,0,0,0
04 --> term3120 Nt=2 TF=2 maxTF=1 @{0 34 0} TFf=0,0,0,0,0,0,2
06 --> term2966 Nt=3 TF=3 maxTF=1 @{0 38 6} TFf=0,0,0,0,0,0,3
0auth2 --> term4739 Nt=1 TF=1 maxTF=1 @{0 47 0} TFf=0,1,0,0,0,0,0
1 --> term1074 Nt=49 TF=50 maxTF=2 @{0 50 6} TFf=0,10,0,0,2,0,38
10 --> term5251 Nt=1 TF=1 maxTF=1 @{0 153 6} TFf=1,0,0,0,0,0,0
100 --> term4546 Nt=1 TF=1 maxTF=1 @{0 157 4} TFf=0,1,0,0,0,0,0
1034 --> term4303 Nt=1 TF=1 maxTF=1 @{0 161 2} TFf=0,1,0,0,0,0,0
11 --> term4947 Nt=1 TF=1 maxTF=1 @{0 165 0} TFf=0,1,0,0,0,0,0
11n --> term1211 Nt=1 TF=1 maxTF=1 @{0 168 6} TFf=0,1,0,0,0,0,0
12 --> term3246 Nt=1 TF=1 maxTF=1 @{0 171 6} TFf=

In [113]:
term_dict = {}

for kv in index.getLexicon():
    term_dict[kv.getKey()] = kv.getValue().frequency


In [114]:
term_dict_ordered = sorted(term_dict.items(), key= lambda x: x[1], reverse=True)

In [115]:
top_k = 50

term_dict_ordered[:top_k]

[('dblp', 4764),
 ('www', 1958),
 ('org', 1957),
 ('bibsonomi', 1956),
 ('bibtex', 1956),
 ('http', 1956),
 ('null', 636),
 ('genealogi', 404),
 ('system', 229),
 ('dnb', 202),
 ('und', 192),
 ('us', 191),
 ('base', 188),
 ('network', 182),
 ('model', 169),
 ('isda', 155),
 ('sylvest', 155),
 ('der', 146),
 ('algorithm', 144),
 ('dfn', 141),
 ('forum', 141),
 ('equat', 131),
 ('method', 100),
 ('data', 95),
 ('von', 94),
 ('optim', 88),
 ('analysi', 87),
 ('de', 87),
 ('multi', 84),
 ('fuzzi', 83),
 ('design', 78),
 ('approach', 76),
 ('control', 76),
 ('genet', 75),
 ('learn', 75),
 ('die', 71),
 ('problem', 70),
 ('comput', 67),
 ('applic', 66),
 ('effici', 64),
 ('gener', 64),
 ('neural', 64),
 ('hardwar', 62),
 ('intellig', 60),
 ('medizin', 60),
 ('program', 60),
 ('swarm', 59),
 ('time', 59),
 ('gesundheit', 58),
 ('adapt', 57)]

In [116]:
search_engine_tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")

/tmp/ipykernel_26740/3523076512.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine_tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")


In [117]:
search_engine_tf = pt.BatchRetrieve(index, wmodel="Tf")

/tmp/ipykernel_26740/3266775375.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine_tf = pt.BatchRetrieve(index, wmodel="Tf")


In [118]:
search_engine_tf_idf.search("Design")

,qid,docid,docno,rank,score,query
0,1,563,https://www.bibsonomy.org/bibtex/29fdf1d326323...,0,3.528028,Design
1,1,1136,https://www.bibsonomy.org/bibtex/20ce861307edb...,1,2.949077,Design
2,1,1923,https://www.bibsonomy.org/bibtex/2775f2422f38f...,2,2.943166,Design
3,1,405,https://www.bibsonomy.org/bibtex/2bde0185914b3...,3,2.870834,Design
4,1,406,https://www.bibsonomy.org/bibtex/2be1b2c3ee1fe...,4,2.870834,Design
...,...,...,...,...,...,...
71,1,1353,https://www.bibsonomy.org/bibtex/296a874a14a23...,71,2.350853,Design
72,1,1442,https://www.bibsonomy.org/bibtex/204f616bea672...,72,2.304476,Design
73,1,1639,https://www.bibsonomy.org/bibtex/237e598097ea9...,73,2.304476,Design
74,1,795,https://www.bibsonomy.org/bibtex/2a4e2e8a91a55...,74,2.259892,Design


In [119]:
index.getLexicon()["design"].toString()

'term96 Nt=76 TF=78 maxTF=2 @{0 19655 2} TFf=0,74,2,0,0,0,2'

In [120]:
search_engine_tf.search("Design")

,qid,docid,docno,rank,score,query
0,1,563,https://www.bibsonomy.org/bibtex/29fdf1d326323...,0,2.0,Design
1,1,1136,https://www.bibsonomy.org/bibtex/20ce861307edb...,1,2.0,Design
2,1,10,https://www.bibsonomy.org/bibtex/2416aabb518b0...,2,1.0,Design
3,1,44,https://www.bibsonomy.org/bibtex/2f8753ced7232...,3,1.0,Design
4,1,57,https://www.bibsonomy.org/bibtex/25b70d9c1170b...,4,1.0,Design
...,...,...,...,...,...,...
71,1,1790,https://www.bibsonomy.org/bibtex/2cf7b7d045efb...,71,1.0,Design
72,1,1857,https://www.bibsonomy.org/bibtex/2a30ee7617ce5...,72,1.0,Design
73,1,1885,https://www.bibsonomy.org/bibtex/2f80cf4a443f2...,73,1.0,Design
74,1,1911,https://www.bibsonomy.org/bibtex/254e9edb62546...,74,1.0,Design


In [121]:
pickle.dump(df_jahreswechsel_filtered, open("./jw_index_mult/jw_publications.pkl", "wb"))